# MLP

In [1]:
import numpy as np
import keras
import os

from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras.callbacks import TensorBoard

Using TensorFlow backend.


## Load dataset

In [2]:
max_words = 1000
batch_size = 32
epochs = 5

In [3]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
8982 train sequences
2246 test sequences


In [4]:
data = np.load('/Users/alorozco53/.keras/datasets/reuters.npz')
data['x'].shape, data['y'].shape

((11228,), (11228,))

In [5]:
num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

46 classes


In [6]:
# Vectorize sequence data
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [7]:
# Convert class vector to binary class matrix
# for use with categorical_crossentropy
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (8982, 46)
y_test shape: (2246, 46)


## Model definition

In [8]:
model = Sequential()

model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['mae', 'acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 46)                23598     
_________________________________________________________________
activation_2 (Activation)    (None, 46)                0         
Total params: 536,110
Trainable params: 536,110
Non-trainable params: 0
_________________________________________________________________


In [10]:
# callback config
logdir = 'mlp-logs'
if not os.path.exists(logdir):
    os.makedirs(logdir)
tb = TensorBoard(log_dir=logdir, write_graph=True, histogram_freq=0, write_images=True)

## Training and Evaluation

In [11]:
# Finally, train
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[tb],
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 3s - loss: 1.4290 - mean_absolute_error: 0.0205 - acc: 0.6785 - val_loss: 1.0904 - val_mean_absolute_error: 0.0165 - val_acc: 0.7664
Epoch 2/5
8083/8083 [==============================] - 2s - loss: 0.7891 - mean_absolute_error: 0.0131 - acc: 0.8171 - val_loss: 0.9380 - val_mean_absolute_error: 0.0142 - val_acc: 0.7864
Epoch 3/5
8083/8083 [==============================] - 2s - loss: 0.5495 - mean_absolute_error: 0.0103 - acc: 0.8670 - val_loss: 0.8898 - val_mean_absolute_error: 0.0127 - val_acc: 0.7976
Epoch 4/5
8083/8083 [==============================] - 3s - loss: 0.4159 - mean_absolute_error: 0.0082 - acc: 0.8994 - val_loss: 0.8798 - val_mean_absolute_error: 0.0122 - val_acc: 0.8065
Epoch 5/5
8083/8083 [==============================] - 3s - loss: 0.3260 - mean_absolute_error: 0.0068 - acc: 0.9167 - val_loss: 0.9065 - val_mean_absolute_error: 0.0120 - val_acc: 0.7998


In [12]:
# Evaluation
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

1696/2246 [=====================>........] - ETA: 0sTest score: 0.883881453839
Test accuracy: 0.0116933315797


In [13]:
# Save the model
model_path = os.path.join(logdir, 'model.h5')
model.save(model_path)
print('model saved in:', model_path)

model saved in: mlp-logs/model.h5
